In [1]:
print("hello")

hello


In [21]:
from typing import List
import requests
import numpy as np
import os
import faiss
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer
from sklearn.metrics.pairwise import cosine_similarity

In [19]:


# class CustomEmbeddingModel:
#     def __init__(self, model_name):
#         super().__init__()
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name)
#         self.model = AutoModel.from_pretrained(model_name)
        

#     @staticmethod
#     def save_vectors_to_faiss_db(vectors, database_path):
#         vectors_np = np.array(vectors, dtype=np.float32)
#         # index = faiss.IndexHNSWFlat(vectors_np.shape[1], faiss.METRIC_INNER_PRODUCT)
#         # index = faiss.IndexFlatL2(vectors_np.shape[1])
#         index = faiss.IndexFlatIP(vectors_np.shape[1])
#         faiss.normalize_L2(vectors_np)
#         index.add(vectors_np)
#         os.makedirs(os.path.dirname(database_path), exist_ok=True)
#         try:
#             faiss.write_index(index, database_path)
#             print(f"Faiss database saved successfully to {database_path}")
#         except Exception as e:
#             print(f"Error saving Faiss database: {e}")

#     @staticmethod
#     def encode_text(self,text):
#         inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
#         outputs = self.model(**inputs)
#         vectors = outputs.last_hidden_state.mean(dim=1).tolist()
#         return vectors

#     @staticmethod
#     def load_faissdb(db_path, user_input, segmented_text):
#         index = faiss.read_index(db_path)
#         input_text_vector = CustomEmbeddingModel.encode_text(user_input)
#         input_text_vector = np.array(input_text_vector, dtype=np.float32)
#         distances, indices = index.search(np.array(input_text_vector), k=5)
#         similar_texts = [segmented_text[i] for i in indices[0]]
#         return similar_texts

#     @staticmethod
#     def segment_text(file_contents, chunk_size):
#         tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
#         splitter = HuggingFaceTextSplitter(tokenizer, trim_chunks=False)

#         sentences = splitter.chunks(file_contents, chunk_size)

#         result_chunks = []
#         current_chunk = ""
#         for i in range(len(sentences)):
#             current_chunk += f" {sentences[i]}"
#             if current_chunk.count(" ") >= 256:
#                 result_chunks.append(current_chunk)
#                 current_chunk = f"{sentences[i-2]} {sentences[i-1]} {sentences[i]}"
#         return result_chunks

# with open('docs.txt', 'r', encoding='utf-8') as file:
#     file_contents = file.read()

# segmented_text = CustomEmbeddingModel.segment_text(file_contents, chunk_size=50)

# model_paths = [
#     'sentence-transformers/all-mpnet-base-v2',
#     'sentence-transformers/paraphrase-MiniLM-L6-v2',
#     "BAAI/bge-base-en-v1.5",
#     "BAAI/bge-large-en-v1.5",
#     'sentence-transformers/all-MiniLM-L6-v2',
#     "thenlper/gte-base"  # Add additional model paths if needed
# ]

# no = 1
# for i in model_paths:
#     database_path = "vector_db_test_sd/faiss_store/vectordb_faisstest_" + str(no)
#     embedder = CustomEmbeddingModel(model_name=i)
#     content_embedded = embedder.encode_text(text=segmented_text)
#     CustomEmbeddingModel.save_vectors_to_faiss_db(content_embedded, database_path)
#     no += 1

# # user_input = input("User input: ")
# # context = []
# # count = 1
# # for j in model_paths:
# #     database_path1 = "vector_db_test_sd/faiss_store/vectordb_faisstest_" + str(count)
# #     embedder1 = CustomEmbeddingModel(model_name=j)
# #     userinput_encoded = embedder1.encode_text([user_input])
# #     similar_texts = CustomEmbeddingModel.load_faissdb(database_path1, userinput_encoded, segmented_text)
# #     context.append(similar_texts[0])
# #     count += 1

# # print(context)



TypeError: CustomEmbeddingModel.encode_text() missing 1 required positional argument: 'self'

In [24]:


class CustomEmbeddingModel:
    def __init__(self, model_name):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def save_vectors_to_faiss_db(self, vectors, database_path):
        vectors_np = np.array(vectors, dtype=np.float32)
        index = faiss.IndexFlatIP(vectors_np.shape[1])
        faiss.normalize_L2(vectors_np)
        index.add(vectors_np)
        os.makedirs(os.path.dirname(database_path), exist_ok=True)
        try:
            faiss.write_index(index, database_path)
            print(f"Faiss database saved successfully to {database_path}")
        except Exception as e:
            print(f"Error saving Faiss database: {e}")

    def encode_text(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        outputs = self.model(**inputs)
        vectors = outputs.last_hidden_state.mean(dim=1).tolist()
        return vectors

    def load_faissdb(db_path, user_input, segmented_text):
        index = faiss.read_index(db_path)
        input_text_vector = self.encode_text(user_input)
        input_text_vector = np.array(input_text_vector, dtype=np.float32)
        distances, indices = index.search(np.array(input_text_vector), k=5)
        similar_texts = [segmented_text[i] for i in indices[0]]
        return similar_texts

    def segment_text(file_contents, chunk_size):
        tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
        splitter = HuggingFaceTextSplitter(tokenizer, trim_chunks=False)

        sentences = splitter.chunks(file_contents, chunk_size)

        result_chunks = []
        current_chunk = ""
        for i in range(len(sentences)):
            current_chunk += f" {sentences[i]}"
            if current_chunk.count(" ") >= 256:
                result_chunks.append(current_chunk)
                current_chunk = f"{sentences[i-2]} {sentences[i-1]} {sentences[i]}"
        return result_chunks

with open('docs.txt', 'r', encoding='utf-8') as file:
    file_contents = file.read()

segmented_text = CustomEmbeddingModel.segment_text(file_contents, chunk_size=50)

model_paths = [
    'sentence-transformers/all-mpnet-base-v2',
    'sentence-transformers/paraphrase-MiniLM-L6-v2',
    "BAAI/bge-base-en-v1.5",
    "BAAI/bge-large-en-v1.5",
    'sentence-transformers/all-MiniLM-L6-v2',
    "thenlper/gte-base"  # Add additional model paths if needed
]

no = 1
for i in model_paths:
    database_path = "vector_db_test_sd/faiss_store/vectordb_faisstest_" + str(no)
    embedder = CustomEmbeddingModel(model_name=i)
    content_embedded = embedder.encode_text(text=segmented_text)
    embedder.save_vectors_to_faiss_db(content_embedded, database_path)
    no += 1


Faiss database saved successfully to vector_db_test_sd/faiss_store/vectordb_faisstest_1
Faiss database saved successfully to vector_db_test_sd/faiss_store/vectordb_faisstest_2
Faiss database saved successfully to vector_db_test_sd/faiss_store/vectordb_faisstest_3
Faiss database saved successfully to vector_db_test_sd/faiss_store/vectordb_faisstest_4
Faiss database saved successfully to vector_db_test_sd/faiss_store/vectordb_faisstest_5
Faiss database saved successfully to vector_db_test_sd/faiss_store/vectordb_faisstest_6


In [ ]:
import requests
class CustomEmbeddingModel:
    def __init__(self, model_name):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    # def save_vectors_to_faiss_db(self, vectors, database_path):
    #     vectors_np = np.array(vectors, dtype=np.float32)
    #     index = faiss.IndexFlatIP(vectors_np.shape[1])
    #     faiss.normalize_L2(vectors_np)
    #     index.add(vectors_np)
    #     os.makedirs(os.path.dirname(database_path), exist_ok=True)
    #     try:
    #         faiss.write_index(index, database_path)
    #         print(f"Faiss database saved successfully to {database_path}")
    #     except Exception as e:
    #         print(f"Error saving Faiss database: {e}")

    def encode_text(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        outputs = self.model(**inputs)
        vectors = outputs.last_hidden_state.mean(dim=1).tolist()
        return vectors

    def load_faissdb(self,db_path, user_input, segmented_text):
        index = faiss.read_index(db_path)
        input_text_vector = self.encode_text(user_input)
        input_text_vector = np.array(input_text_vector, dtype=np.float32)
        distances, indices = index.search(np.array(input_text_vector), k=3)
        similar_texts = [segmented_text[i] for i in indices[0]]
        return similar_texts

    # def segment_text(file_contents, chunk_size):
    #     tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
    #     splitter = HuggingFaceTextSplitter(tokenizer, trim_chunks=False)

    #     sentences = splitter.chunks(file_contents, chunk_size)

    #     result_chunks = []
    #     current_chunk = ""
    #     for i in range(len(sentences)):
    #         current_chunk += f" {sentences[i]}"
    #         if current_chunk.count(" ") >= 256:
    #             result_chunks.append(current_chunk)
    #             current_chunk = f"{sentences[i-2]} {sentences[i-1]} {sentences[i]}"
    #     return result_chunks


user_input = input("User input: ")
context = []
count = 1
for j in model_paths:
    database_path1 = "vector_db_test_sd/faiss_store/vectordb_faisstest_" + str(count)
    embedder1 = CustomEmbeddingModel(model_name=j)
    similar_texts = embedder1.load_faissdb(database_path1,user_input,segmented_text)
    context.append(similar_texts[0])
    count += 1

print(context)

# data = set(context)
final_context = " ".join(context)
print(final_context)
print("##################################################################################")

payload = {
  "message_content": user_input,
  "content": final_context,
  "company_name": "uplands rehabilitation",
  "domain": "care center"
}

response = requests.post("http://46.43.144.145:7010/rulebased_bot",json=payload)

print(response.json())

In [ ]:
Contact numbers are provided throughout the context for various locations and services.

In [ ]:
We have provided various phone numbers throughout the context. Please refer to them for specific contact details you need.

In [ ]:
We will provide you with the necessary contact details when we are corresponding with you or providing services. If needed, we may ask for your name, email address, phone number, and other information to ensure smooth communication.